#Download Dataset

In [ ]:
!wget https://nextcloud.in.tum.de/index.php/s/9tp3yCSiLgjLWdj/download/liver_endoscopy_dataset.zip

--2024-06-09 16:09:49--  https://nextcloud.in.tum.de/index.php/s/9tp3yCSiLgjLWdj/download/liver_endoscopy_dataset.zip
Resolving nextcloud.in.tum.de (nextcloud.in.tum.de)... 131.159.2.68, 2a09:80c0:2::68
Connecting to nextcloud.in.tum.de (nextcloud.in.tum.de)|131.159.2.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2907480775 (2.7G) [application/zip]
Saving to: ‘liver_endoscopy_dataset.zip’

liver_endoscopy_dat 100%[===================>]   2.71G  23.7MB/s    in 2m 3s   

2024-06-09 16:11:53 (22.5 MB/s) - ‘liver_endoscopy_dataset.zip’ saved [2907480775/2907480775]



#Prepare Dataset

In [ ]:
  !unzip -qq liver_endoscopy_dataset
  !rm liver_endoscopy_dataset.zip

#Install Additional Requirements

In [ ]:
!pip -qq install pytorch_lightning==1.6.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.9/582.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 56.8 MB/s eta 0:00:00


#Imports

In [ ]:
import json
from collections import defaultdict
from pathlib import Path
import random
from typing import Optional

import torch
from torch import nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import pytorch_lightning as pl
pl.seed_everything(42)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


42

#Dataset
We load the liver endoscopy dataset here. This cell defines the video splits, correctly loads the dataset depending on the task.

In [ ]:

video_splits = {'train': ['01', '09', '17', '20', '25', '27', '28', '35', '37'], 'val': ['12','18', '24'], 'test': ['26', '48', '52', '43', '55']}

class LiverEndoscopy(Dataset):
    def __init__(self,
                 task_type: str = 'classification', split: str = 'train', balance_data: bool = False, temporal: bool = False,
                 pil_transform: Optional[transforms.Compose] = None, tensor_transform: Optional[transforms.Compose] = None):
        assert split in ['train', 'val', 'test']
        self.split = split
        self.balance_data = balance_data
        self.task_type = task_type
        self.temporal = temporal
        self.pil_transform = pil_transform
        self.tensor_transform = tensor_transform

        export_dataset_path = Path('data')
        self.images_path = export_dataset_path / 'images'
        self.seg_masks_path = export_dataset_path / 'seg_masks'
        with open(export_dataset_path / 'classification_annotations.json', 'r') as f:
            self.classification_annotations = json.load(f)
        with open(export_dataset_path / 'phase_annotations.json', 'r') as f:
            self.workflow_phase_annotations = json.load(f)
        with open(export_dataset_path / 'has_liver.json', 'r') as f:
            self.has_liver = json.load(f)

        if task_type == 'classification' or task_type == 'segmentation' or (task_type == 'workflow' and not temporal):
            self.image_names = []
            for image_path in sorted(self.images_path.glob('*.png')):
                video_id = image_path.name.split('_')[0].replace('video', '')
                if video_id in video_splits[split]:
                    self.image_names.append(image_path.name.replace('.png', ''))
            self.image_names = sorted(self.image_names)

        elif task_type == 'workflow' and temporal:
            self.window_size = 8
            self.downsample_factor = 5
            chunks = []
            start = -1
            end = -1
            prev_frame_number = None
            prev_video_id = None
            for image_path in sorted(self.images_path.glob('*.png')):
                video_id = image_path.name.split('_')[0].replace('video', '')
                if video_id not in video_splits[split]:
                    continue
                if prev_video_id is None:
                    prev_video_id = video_id
                elif prev_video_id != video_id:
                    chunks.append((prev_video_id, start, end))
                    start = -1
                    end = -1
                    prev_frame_number = None
                    prev_video_id = video_id

                frame_number = int(image_path.name.split('_')[1].replace('.png', ''))
                if prev_frame_number is None:
                    start = frame_number
                else:
                    if frame_number == prev_frame_number + 1:
                        end = frame_number
                    else:
                        chunks.append((video_id, start, end))
                        start = frame_number
                prev_frame_number = frame_number
            self.windows = []
            for video_id, start, end in chunks:
                for last_frame_index in range(start + self.window_size, end + 1):
                    all_frames = []
                    for frame_index in range(last_frame_index - self.window_size * self.downsample_factor, last_frame_index, self.downsample_factor):
                        if frame_index < start or frame_index > end:
                            continue
                        all_frames.append(frame_index)
                    if len(all_frames) == self.window_size:
                        self.windows.append((video_id, all_frames))

        if balance_data:
            self.do_balance_data(task_type, temporal)

    def do_balance_data(self, task_type, temporal):
        print('Balancing data by oversampling under-represented classes...')
        class_to_samples = defaultdict(list)
        if not temporal:
            for image_name in self.image_names:
                if task_type == 'classification':
                    label = self.classification_annotations[image_name]
                elif task_type == 'segmentation':
                    label = self.has_liver[image_name]
                elif task_type == 'workflow' and not temporal:
                    label = self.workflow_phase_annotations[image_name]
                class_to_samples[label].append(image_name)
            max_number = max([len(elem) for elem in class_to_samples.values()])
            self.image_names = []
            for key, value in class_to_samples.items():
                if len(value) < max_number:
                    self.image_names += random.choices(value, k=max_number)
                else:
                    self.image_names += value
            random.shuffle(self.image_names)
        else:
            for video_id, window in self.windows:
                label = self.workflow_phase_annotations[f'video{video_id}_{str(window[-1]).zfill(6)}']
                class_to_samples[label].append((video_id, window))
            max_number = max([len(elem) for elem in class_to_samples.values()])
            self.windows = []
            for key, value in class_to_samples.items():
                if len(value) < max_number:
                    self.windows += random.choices(value, k=max_number)
                else:
                    self.windows += value
            random.shuffle(self.windows)

    def __len__(self):
        if self.task_type == 'workflow' and self.temporal:
            return len(self.windows)
        else:
            return len(self.image_names)

    def phase_label_to_number(self, phase_label):
        if phase_label == 'Preparation':
            return 0
        elif phase_label == 'CalotTriangleDissection':
            return 1
        elif phase_label == 'GallbladderDissection':
            return 2
        else:
            raise ValueError('Unknown phase label: {}'.format(phase_label))

    def number_to_phase_label(self, phase_number):
        if phase_number == 0:
            return 'Preparation'
        elif phase_number == 1:
            return 'CalotTriangleDissection'
        elif phase_number == 2:
            return 'GallbladderDissection'
        else:
            raise ValueError('Unknown phase number: {}'.format(phase_number))

    def __getitem__(self, index):
        if self.task_type == 'workflow' and self.temporal:
            video_id, window = self.windows[index]
            image_paths = []
            for frame_number in window:
                image_paths.append(self.images_path / f'video{video_id}_{str(frame_number).zfill(6)}.png')

            image_tensors = []
            for image_path in image_paths:
                image = Image.open(image_path)
                if self.pil_transform is not None:
                    image = self.pil_transform(image)
                image_tensor = transforms.ToTensor()(image)
                if self.tensor_transform is not None:
                    image_tensor = self.tensor_transform(image_tensor)
                image_tensors.append(image_tensor)

            phase = self.phase_label_to_number(self.workflow_phase_annotations[image_paths[-1].name.replace('.png', '')])
            image_tensors = torch.stack(image_tensors)
            return image_tensors, phase
        else:
            image_name = self.image_names[index]
            image_path = self.images_path / f'{image_name}.png'
            seg_mask_path = self.seg_masks_path / f'{image_name}_liver_mask.png'
            image = Image.open(image_path)
            if self.pil_transform is not None:
                image = self.pil_transform(image)
            image_tensor = transforms.ToTensor()(image)
            if self.tensor_transform is not None:
                image_tensor = self.tensor_transform(image_tensor)
            seg_mask = Image.open(seg_mask_path)
            if self.pil_transform is not None:
                seg_mask = self.pil_transform(seg_mask)
            seg_mask_tensor = transforms.ToTensor()(seg_mask)[0].float()
            if self.tensor_transform is not None:
                seg_mask_tensor = self.tensor_transform(seg_mask_tensor)
            intrument_exists = int(self.classification_annotations[image_name])
            phase = self.phase_label_to_number(self.workflow_phase_annotations[image_name])

            if self.task_type == 'classification':
                return image_tensor, intrument_exists
            elif self.task_type == 'segmentation':
                return image_tensor, seg_mask_tensor
            elif self.task_type == 'workflow' and not self.temporal:
                return image_tensor, phase


# Classification 1
###### You need to define classification model with two CNN layers, ReLU and pooling operator afterwards and three Linear layers.

In [ ]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super().__init__()
        #######################################
        # To be filled
        #
        # define the layers
        # conv1: 3 filters, kernel_size = 5
        # maxpooling: size = 2
        # conv1: 6 filters, kernel_size = 5
        # fc1: output features = 120
        # fc2: output features = 84
        # fc3: output features = num_classes
        #######################################


        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Classification 2
###### You need to define classification model with pretrained ResNet18.

In [ ]:
# class ResNetModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         #######################################
#         # To be filled
#         #
#         # define the backbone using ResNet18
#         #######################################
#         self.image_backbone = None
#         #######################################
#         # To be filled
#         #
#         # disable the last layer, redefine it to match the number of classes
#         #######################################
#         self.image_backbone.fc = None
#         self.linear = None

#     def forward(self, x):
#         x = self.image_backbone(x)
#         x = self.linear(x)
#         return x

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNetModel(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNetModel, self).__init__()
        #######################################
        # define the backbone using ResNet18
        #######################################
        self.image_backbone = models.resnet18(pretrained=True)
        #######################################
        # disable the last layer, redefine it to match the number of classes
        #######################################
        num_features = self.image_backbone.fc.in_features
        self.image_backbone.fc = nn.Identity()  # Disable the last layer
        self.linear = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.image_backbone(x)
        x = self.linear(x)
        return x


# Classification 3
###### Train both models with CrossEntropy loss.

In [ ]:
# from sklearn.metrics import classification_report
# import numpy as np
# import matplotlib.pyplot as plt
# import torch.nn as nn


# class Classifier(pl.LightningModule):
#     def __init__(self):
#         super().__init__()
#         #######################################
#         # To be filled
#         #
#         # use the model from Classification 1
#         # define the loss
#         #######################################
#         self.model = None
#         # define the loss
#         self.loss = None

#         self.train_preds = []
#         self.train_gts = []
#         self.val_preds = []
#         self.val_gts = []
#         self.test_preds = []
#         self.test_gts = []
#         self.reset_metrics()

#         self.train_loss = []
#         self.val_loss = []
#         self.test_loss = []

#     def training_step(self, batch, batch_idx):
#         #######################################
#         # To be filled
#         #
#         # run the model on input image to get the prediction y_hat
#         # specify the loss inputs
#         #######################################
#         x, y = batch
#         y_hat = None
#         loss = None

#         self.update_metrics(y, y_hat, split='train')
#         self.train_loss.append(loss.item())
#         return {'loss': loss}


#     def validation_step(self, batch, batch_idx):
#         # validation_step defines the validation loop.
#         x, y = batch
#         #######################################
#         # To be filled
#         #
#         # run the model on input image to get the prediction y_hat
#         # specify the loss inputs
#         #######################################
#         y_hat = None
#         loss = None
#         self.update_metrics(y, y_hat, split='val')
#         self.val_loss.append(loss.item())
#         return {'val_loss': loss}

#     def test_step(self, batch, batch_idx):
#         # test_step defines the test loop.
#         x, y = batch
#         #######################################
#         # To be filled
#         #
#         # run the model on input image to get the prediction y_hat
#         # specify the loss inputs
#         #######################################
#         y_hat = None
#         loss = None
#         self.update_metrics(y, y_hat, split='test')
#         self.test_loss.append(loss.item())
#         return {'test_loss': loss}

#     def configure_optimizers(self):
#         #######################################
#         # To be filled
#         #
#         # Define Adam optimizer with different learning rates in range (1e-2, 1e-4)
#         #######################################
#         optimizer = None
#         return optimizer

#     def reset_metrics(self, split=None):
#         if split == 'train':
#             self.train_preds = []
#             self.train_gts = []
#         elif split == 'val':
#             self.val_preds = []
#             self.val_gts = []
#         elif split == 'test':
#             self.test_preds = []
#             self.test_gts = []
#         else:
#             self.train_preds = []
#             self.train_gts = []
#             self.val_preds = []
#             self.val_gts = []
#             self.test_preds = []
#             self.test_gts = []

#     def update_metrics(self, gt, pred, split='train'):
#         if split == 'train':
#             self.train_preds.extend(pred.detach().cpu().numpy().argmax(1))
#             self.train_gts.extend(gt.detach().cpu().numpy())
#         elif split == 'val':
#             self.val_preds.extend(pred.detach().cpu().numpy().argmax(1))
#             self.val_gts.extend(gt.detach().cpu().numpy())
#         elif split == 'test':
#             self.test_preds.extend(pred.detach().cpu().numpy().argmax(1))
#             self.test_gts.extend(gt.detach().cpu().numpy())
#         else:
#             raise NotImplementedError()

#     def training_epoch_end(self, outputs):
#         self.evaluate_predictions(split='train')
#         self.reset_metrics(split='train')

#     def validation_epoch_end(self, outputs):
#         self.evaluate_predictions(split='val')
#         self.reset_metrics(split='val')

#     def test_epoch_end(self, outputs):
#         self.evaluate_predictions(split='test')
#         self.reset_metrics(split='test')

#     def evaluate_predictions(self, split):
#         if split == 'train':
#             preds = self.train_preds
#             gts = self.train_gts
#         elif split == 'val':
#             preds = self.val_preds
#             gts = self.val_gts
#         elif split == 'test':
#             preds = self.test_preds
#             gts = self.test_gts
#         else:
#             raise NotImplementedError()

#         cls_report = classification_report(gts, preds)
#         print(split)
#         print(cls_report)




In [ ]:
# from sklearn.metrics import classification_report
# import numpy as np
# import matplotlib.pyplot as plt
# import torch.nn as nn


# class Classifier(pl.LightningModule):
#     def __init__(self):
#         super().__init__()
#         #######################################
#         # To be filled
#         #
#         # use the Resnet model from Classification 2
#         # define the loss
#         #######################################
#         self.model = None
#         # define the loss
#         self.loss = None

#         self.train_preds = []
#         self.train_gts = []
#         self.val_preds = []
#         self.val_gts = []
#         self.test_preds = []
#         self.test_gts = []
#         self.reset_metrics()

#         self.train_loss = []
#         self.val_loss = []
#         self.test_loss = []

#     def training_step(self, batch, batch_idx):
#         #######################################
#         # To be filled
#         #
#         # run the model on input image to get the prediction y_hat
#         # specify the loss inputs
#         #######################################
#         x, y = batch
#         y_hat = None
#         loss = None

#         self.update_metrics(y, y_hat, split='train')
#         self.train_loss.append(loss.item())
#         return {'loss': loss}


#     def validation_step(self, batch, batch_idx):
#         # validation_step defines the validation loop.
#         x, y = batch
#         #######################################
#         # To be filled
#         #
#         # run the model on input image to get the prediction y_hat
#         # specify the loss inputs
#         #######################################
#         y_hat = None
#         loss = None
#         self.update_metrics(y, y_hat, split='val')
#         self.val_loss.append(loss.item())
#         return {'val_loss': loss}

#     def test_step(self, batch, batch_idx):
#         # test_step defines the test loop.
#         x, y = batch
#         #######################################
#         # To be filled
#         #
#         # run the model on input image to get the prediction y_hat
#         # specify the loss inputs
#         #######################################
#         y_hat = None
#         loss = None
#         self.update_metrics(y, y_hat, split='test')
#         self.test_loss.append(loss.item())
#         return {'test_loss': loss}

#     def configure_optimizers(self):
#         #######################################
#         # To be filled
#         #
#         # Define Adam optimizer with different learning rates in range (1e-2, 1e-4)
#         #######################################
#         optimizer = None
#         return optimizer

#     def reset_metrics(self, split=None):
#         if split == 'train':
#             self.train_preds = []
#             self.train_gts = []
#         elif split == 'val':
#             self.val_preds = []
#             self.val_gts = []
#         elif split == 'test':
#             self.test_preds = []
#             self.test_gts = []
#         else:
#             self.train_preds = []
#             self.train_gts = []
#             self.val_preds = []
#             self.val_gts = []
#             self.test_preds = []
#             self.test_gts = []

#     def update_metrics(self, gt, pred, split='train'):
#         if split == 'train':
#             self.train_preds.extend(pred.detach().cpu().numpy().argmax(1))
#             self.train_gts.extend(gt.detach().cpu().numpy())
#         elif split == 'val':
#             self.val_preds.extend(pred.detach().cpu().numpy().argmax(1))
#             self.val_gts.extend(gt.detach().cpu().numpy())
#         elif split == 'test':
#             self.test_preds.extend(pred.detach().cpu().numpy().argmax(1))
#             self.test_gts.extend(gt.detach().cpu().numpy())
#         else:
#             raise NotImplementedError()

#     def training_epoch_end(self, outputs):
#         self.evaluate_predictions(split='train')
#         self.reset_metrics(split='train')

#     def validation_epoch_end(self, outputs):
#         self.evaluate_predictions(split='val')
#         self.reset_metrics(split='val')

#     def test_epoch_end(self, outputs):
#         self.evaluate_predictions(split='test')
#         self.reset_metrics(split='test')

#     def evaluate_predictions(self, split):
#         if split == 'train':
#             preds = self.train_preds
#             gts = self.train_gts
#         elif split == 'val':
#             preds = self.val_preds
#             gts = self.val_gts
#         elif split == 'test':
#             preds = self.test_preds
#             gts = self.test_gts
#         else:
#             raise NotImplementedError()

#         cls_report = classification_report(gts, preds)
#         print(split)
#         print(cls_report)





In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

class Classifier(pl.LightningModule):
    def __init__(self, model_type='custom', num_classes=3, learning_rate=1e-3):
        super(Classifier, self).__init__()
        #######################################
        # Define the model
        #######################################
        if model_type == 'custom':
            self.model = ClassificationModel()
        elif model_type == 'resnet':
            self.model = ResNetModel(num_classes=num_classes)
        else:
            raise ValueError("Invalid model type. Choose 'custom' or 'resnet'.")

        # Define the loss
        self.loss = nn.CrossEntropyLoss()

        self.train_preds = []
        self.train_gts = []
        self.val_preds = []
        self.val_gts = []
        self.test_preds = []
        self.test_gts = []
        self.reset_metrics()

        self.train_loss = []
        self.val_loss = []
        self.test_loss = []

        self.learning_rate = learning_rate

    def training_step(self, batch, batch_idx):
        #######################################
        # Run the model on input image to get the prediction y_hat
        # Specify the loss inputs
        #######################################
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)

        self.update_metrics(y, y_hat, split='train')
        self.train_loss.append(loss.item())
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        # Validation step defines the validation loop.
        x, y = batch
        #######################################
        # Run the model on input image to get the prediction y_hat
        # Specify the loss inputs
        #######################################
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.update_metrics(y, y_hat, split='val')
        self.val_loss.append(loss.item())
        return {'val_loss': loss}

    def test_step(self, batch, batch_idx):
        # Test step defines the test loop.
        x, y = batch
        #######################################
        # Run the model on input image to get the prediction y_hat
        # Specify the loss inputs
        #######################################
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.update_metrics(y, y_hat, split='test')
        self.test_loss.append(loss.item())
        return {'test_loss': loss}

    def configure_optimizers(self):
        #######################################
        # Define Adam optimizer with different learning rates in range (1e-2, 1e-4)
        #######################################
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

    def reset_metrics(self, split=None):
        if split == 'train':
            self.train_preds = []
            self.train_gts = []
        elif split == 'val':
            self.val_preds = []
            self.val_gts = []
        elif split == 'test':
            self.test_preds = []
            self.test_gts = []
        else:
            self.train_preds = []
            self.train_gts = []
            self.val_preds = []
            self.val_gts = []
            self.test_preds = []
            self.test_gts = []

    def update_metrics(self, gt, pred, split='train'):
        if split == 'train':
            self.train_preds.extend(pred.detach().cpu().numpy().argmax(1))
            self.train_gts.extend(gt.detach().cpu().numpy())
        elif split == 'val':
            self.val_preds.extend(pred.detach().cpu().numpy().argmax(1))
            self.val_gts.extend(gt.detach().cpu().numpy())
        elif split == 'test':
            self.test_preds.extend(pred.detach().cpu().numpy().argmax(1))
            self.test_gts.extend(gt.detach().cpu().numpy())
        else:
            raise NotImplementedError()

    def training_epoch_end(self, outputs):
        self.evaluate_predictions(split='train')
        self.reset_metrics(split='train')

    def validation_epoch_end(self, outputs):
        self.evaluate_predictions(split='val')
        self.reset_metrics(split='val')

    def test_epoch_end(self, outputs):
        self.evaluate_predictions(split='test')
        self.reset_metrics(split='test')

    def evaluate_predictions(self, split):
        if split == 'train':
            preds = self.train_preds
            gts = self.train_gts
        elif split == 'val':
            preds = self.val_preds
            gts = self.val_gts
        elif split == 'test':
            preds = self.test_preds
            gts = self.test_gts
        else:
            raise NotImplementedError()

        cls_report = classification_report(gts, preds)
        print(split)
        print(cls_report)


In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

class ResNetModel(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNetModel, self).__init__()
        #######################################
        # define the backbone using ResNet18
        #######################################
        self.image_backbone = models.resnet18(pretrained=True)
        #######################################
        # disable the last layer, redefine it to match the number of classes
        #######################################
        num_features = self.image_backbone.fc.in_features
        self.image_backbone.fc = nn.Identity()  # Disable the last layer
        self.linear = nn.Linear(num_features, num_classes)

    def forward(self, x):
        x = self.image_backbone(x)
        x = self.linear(x)
        return x

class Classifier(pl.LightningModule):
    def __init__(self, num_classes=3, learning_rate=1e-3):
        super(Classifier, self).__init__()
        #######################################
        # Use the Resnet model from Classification 2
        #######################################
        self.model = ResNetModel(num_classes=num_classes)
        # Define the loss
        self.loss = nn.CrossEntropyLoss()

        self.train_preds = []
        self.train_gts = []
        self.val_preds = []
        self.val_gts = []
        self.test_preds = []
        self.test_gts = []
        self.reset_metrics()

        self.train_loss = []
        self.val_loss = []
        self.test_loss = []

        self.learning_rate = learning_rate

    def training_step(self, batch, batch_idx):
        #######################################
        # Run the model on input image to get the prediction y_hat
        # Specify the loss inputs
        #######################################
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)

        self.update_metrics(y, y_hat, split='train')
        self.train_loss.append(loss.item())
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        # Validation step defines the validation loop.
        x, y = batch
        #######################################
        # Run the model on input image to get the prediction y_hat
        # Specify the loss inputs
        #######################################
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.update_metrics(y, y_hat, split='val')
        self.val_loss.append(loss.item())
        return {'val_loss': loss}

    def test_step(self, batch, batch_idx):
        # Test step defines the test loop.
        x, y = batch
        #######################################
        # Run the model on input image to get the prediction y_hat
        # Specify the loss inputs
        #######################################
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.update_metrics(y, y_hat, split='test')
        self.test_loss.append(loss.item())
        return {'test_loss': loss}

    def configure_optimizers(self):
        #######################################
        # Define Adam optimizer with different learning rates in range (1e-2, 1e-4)
        #######################################
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

    def reset_metrics(self, split=None):
        if split == 'train':
            self.train_preds = []
            self.train_gts = []
        elif split == 'val':
            self.val_preds = []
            self.val_gts = []
        elif split == 'test':
            self.test_preds = []
            self.test_gts = []
        else:
            self.train_preds = []
            self.train_gts = []
            self.val_preds = []
            self.val_gts = []
            self.test_preds = []
            self.test_gts = []

    def update_metrics(self, gt, pred, split='train'):
        if split == 'train':
            self.train_preds.extend(pred.detach().cpu().numpy().argmax(1))
            self.train_gts.extend(gt.detach().cpu().numpy())
        elif split == 'val':
            self.val_preds.extend(pred.detach().cpu().numpy().argmax(1))
            self.val_gts.extend(gt.detach().cpu().numpy())
        elif split == 'test':
            self.test_preds.extend(pred.detach().cpu().numpy().argmax(1))
            self.test_gts.extend(gt.detach().cpu().numpy())
        else:
            raise NotImplementedError()

    def training_epoch_end(self, outputs):
        self.evaluate_predictions(split='train')
        self.reset_metrics(split='train')

    def validation_epoch_end(self, outputs):
        self.evaluate_predictions(split='val')
        self.reset_metrics(split='val')

    def test_epoch_end(self, outputs):
        self.evaluate_predictions(split='test')
        self.reset_metrics(split='test')

    def evaluate_predictions(self, split):
        if split == 'train':
            preds = self.train_preds
            gts = self.train_gts
        elif split == 'val':
            preds = self.val_preds
            gts = self.val_gts
        elif split == 'test':
            preds = self.test_preds
            gts = self.test_gts
        else:
            raise NotImplementedError()

        cls_report = classification_report(gts, preds)
        print(split)
        print(cls_report)



# Dataloaders
###### Define the validation and test datasets simular to training dataset.



In [ ]:
pil_transform = transforms.Compose([transforms.Resize((224, 224))])
train_dataset = LiverEndoscopy(split='train', balance_data=True, task_type='classification', temporal=False, pil_transform=pil_transform)
#######################################
# To be filled
# Define the validation and test datasets
# simular to training dataset
#######################################
val_dataset = None
test_dataset = None
#######################################
# To be filled
# Define loaders with batch_size = 64 and
# shuffle=True for train loader and
# shuffle=False for the rest, num_workers=1
#######################################
train_loader = None
val_loader = None
test_loader = None
#######################################
# To be filled
# Define the model
#######################################
model = None

In [ ]:
import json
from collections import defaultdict
from pathlib import Path
import random
from typing import Optional

import torch
from torch import nn
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import pytorch_lightning as pl
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

pl.seed_everything(42)

# Define the LiverEndoscopy class as provided before
video_splits = {'train': ['01', '09', '17', '20', '25', '27', '28', '35', '37'], 'val': ['12','18', '24'], 'test': ['26', '48', '52', '43', '55']}

class LiverEndoscopy(Dataset):
    def __init__(self,
                 task_type: str = 'classification', split: str = 'train', balance_data: bool = False, temporal: bool = False,
                 pil_transform: Optional[transforms.Compose] = None, tensor_transform: Optional[transforms.Compose] = None):
        assert split in ['train', 'val', 'test']
        self.split = split
        self.balance_data = balance_data
        self.task_type = task_type
        self.temporal = temporal
        self.pil_transform = pil_transform
        self.tensor_transform = tensor_transform

        export_dataset_path = Path('data')
        self.images_path = export_dataset_path / 'images'
        self.seg_masks_path = export_dataset_path / 'seg_masks'
        with open(export_dataset_path / 'classification_annotations.json', 'r') as f:
            self.classification_annotations = json.load(f)
        with open(export_dataset_path / 'phase_annotations.json', 'r') as f:
            self.workflow_phase_annotations = json.load(f)
        with open(export_dataset_path / 'has_liver.json', 'r') as f:
            self.has_liver = json.load(f)

        if task_type == 'classification' or task_type == 'segmentation' or (task_type == 'workflow' and not temporal):
            self.image_names = []
            for image_path in sorted(self.images_path.glob('*.png')):
                video_id = image_path.name.split('_')[0].replace('video', '')
                if video_id in video_splits[split]:
                    self.image_names.append(image_path.name.replace('.png', ''))
            self.image_names = sorted(self.image_names)

        elif task_type == 'workflow' and temporal:
            self.window_size = 8
            self.downsample_factor = 5
            chunks = []
            start = -1
            end = -1
            prev_frame_number = None
            prev_video_id = None
            for image_path in sorted(self.images_path.glob('*.png')):
                video_id = image_path.name.split('_')[0].replace('video', '')
                if video_id not in video_splits[split]:
                    continue
                if prev_video_id is None:
                    prev_video_id = video_id
                elif prev_video_id != video_id:
                    chunks.append((prev_video_id, start, end))
                    start = -1
                    end = -1
                    prev_frame_number = None
                    prev_video_id = video_id

                frame_number = int(image_path.name.split('_')[1].replace('.png', ''))
                if prev_frame_number is None:
                    start = frame_number
                else:
                    if frame_number == prev_frame_number + 1:
                        end = frame_number
                    else:
                        chunks.append((video_id, start, end))
                        start = frame_number
                prev_frame_number = frame_number
            self.windows = []
            for video_id, start, end in chunks:
                for last_frame_index in range(start + self.window_size, end + 1):
                    all_frames = []
                    for frame_index in range(last_frame_index - self.window_size * self.downsample_factor, last_frame_index, self.downsample_factor):
                        if frame_index < start or frame_index > end:
                            continue
                        all_frames.append(frame_index)
                    if len(all_frames) == self.window_size:
                        self.windows.append((video_id, all_frames))

        if balance_data:
            self.do_balance_data(task_type, temporal)

    def do_balance_data(self, task_type, temporal):
        print('Balancing data by oversampling under-represented classes...')
        class_to_samples = defaultdict(list)
        if not temporal:
            for image_name in self.image_names:
                if task_type == 'classification':
                    label = self.classification_annotations[image_name]
                elif task_type == 'segmentation':
                    label = self.has_liver[image_name]
                elif task_type == 'workflow' and not temporal:
                    label = self.workflow_phase_annotations[image_name]
                class_to_samples[label].append(image_name)
            max_number = max([len(elem) for elem in class_to_samples.values()])
            self.image_names = []
            for key, value in class_to_samples.items():
                if len(value) < max_number:
                    self.image_names += random.choices(value, k=max_number)
                else:
                    self.image_names += value
            random.shuffle(self.image_names)
        else:
            for video_id, window in self.windows:
                label = self.workflow_phase_annotations[f'video{video_id}_{str(window[-1]).zfill(6)}']
                class_to_samples[label].append((video_id, window))
            max_number = max([len(elem) for elem in class_to_samples.values()])
            self.windows = []
            for key, value in class_to_samples.items():
                if len(value) < max_number:
                    self.windows += random.choices(value, k=max_number)
                else:
                    self.windows += value
            random.shuffle(self.windows)

    def __len__(self):
        if self.task_type == 'workflow' and self.temporal:
            return len(self.windows)
        else:
            return len(self.image_names)

    def phase_label_to_number(self, phase_label):
        if phase_label == 'Preparation':
            return 0
        elif phase_label == 'CalotTriangleDissection':
            return 1
        elif phase_label == 'GallbladderDissection':
            return 2
        else:
            raise ValueError('Unknown phase label: {}'.format(phase_label))

    def number_to_phase_label(self, phase_number):
        if phase_number == 0:
            return 'Preparation'
        elif phase_number == 1:
            return 'CalotTriangleDissection'
        elif phase_number == 2:
            return 'GallbladderDissection'
        else:
            raise ValueError('Unknown phase number: {}'.format(phase_number))

    def __getitem__(self, index):
        if self.task_type == 'workflow' and self.temporal:
            video_id, window = self.windows[index]
            image_paths = []
            for frame_number in window:
                image_paths.append(self.images_path / f'video{video_id}_{str(frame_number).zfill(6)}.png')

            image_tensors = []
            for image_path in image_paths:
                image = Image.open(image_path)
                if self.pil_transform is not None:
                    image = self.pil_transform(image)
                image_tensor = transforms.ToTensor()(image)
                if self.tensor_transform is not None:
                    image_tensor = self.tensor_transform(image_tensor)
                image_tensors.append(image_tensor)

            phase = self.phase_label_to_number(self.workflow_phase_annotations[image_paths[-1].name.replace('.png', '')])
            image_tensors = torch.stack(image_tensors)
            return image_tensors, phase
        else:
            image_name = self.image_names[index]
            image_path = self.images_path / f'{image_name}.png'
            seg_mask_path = self.seg_masks_path / f'{image_name}_liver_mask.png'
            image = Image.open(image_path)
            if self.pil_transform is not None:
                image = self.pil_transform(image)
            image_tensor = transforms.ToTensor()(image)
            if self.tensor_transform is not None:
                image_tensor = self.tensor_transform(image_tensor)
            seg_mask = Image.open(seg_mask_path)
            if self.pil_transform is not None:
                seg_mask = self.pil_transform(seg_mask)
            seg_mask_tensor = transforms.ToTensor()(seg_mask)[0].float()
            if self.tensor_transform is not None:
                seg_mask_tensor = self.tensor_transform(seg_mask_tensor)
            intrument_exists = int(self.classification_annotations[image_name])
            phase = self.phase_label_to_number(self.workflow_phase_annotations[image_name])

            if self.task_type == 'classification':
                return image_tensor, intrument_exists
            elif self.task_type == 'segmentation':
                return image_tensor, seg_mask_tensor
            elif self.task_type == 'workflow' and not self.temporal:
                return image_tensor, phase

# Define ClassificationModel as provided before
class ClassificationModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define Classifier as provided before
class Classifier(pl.LightningModule):
    def __init__(self, model_type='custom', num_classes=3, learning_rate=1e-3):
        super(Classifier, self).__init__()

        if model_type == 'custom':
            self.model = ClassificationModel()
        elif model_type == 'resnet':
            self.model = ResNetModel(num_classes=num_classes)
        else:
            raise ValueError("Invalid model type. Choose 'custom' or 'resnet'.")

        # Define the loss
        self.loss = nn.CrossEntropyLoss()

        self.train_preds = []
        self.train_gts = []
        self.val_preds = []
        self.val_gts = []
        self.test_preds = []
        self.test_gts = []
        self.reset_metrics()

        self.train_loss = []
        self.val_loss = []
        self.test_loss = []

        self.learning_rate = learning_rate

    def training_step(self, batch, batch_idx):

        x, y = batch
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)

        self.update_metrics(y, y_hat, split='train')
        self.train_loss.append(loss.item())
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        # Validation step defines the validation loop.
        x, y = batch

        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.update_metrics(y, y_hat, split='val')
        self.val_loss.append(loss.item())
        return {'val_loss': loss}

    def test_step(self, batch, batch_idx):
        # Test step defines the test loop.
        x, y = batch

        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        self.update_metrics(y, y_hat, split='test')
        self.test_loss.append(loss.item())
        return {'test_loss': loss}

    def configure_optimizers(self):

        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

    def reset_metrics(self, split=None):
        if split == 'train':
            self.train_preds = []
            self.train_gts = []
        elif split == 'val':
            self.val_preds = []
            self.val_gts = []
        elif split == 'test':
            self.test_preds = []
            self.test_gts = []
        else:
            self.train_preds = []
            self.train_gts = []
            self.val_preds = []
            self.val_gts = []
            self.test_preds = []
            self.test_gts = []

    def update_metrics(self, gt, pred, split='train'):
        if split == 'train':
            self.train_preds.extend(pred.detach().cpu().numpy().argmax(1))
            self.train_gts.extend(gt.detach().cpu().numpy())
        elif split == 'val':
            self.val_preds.extend(pred.detach().cpu().numpy().argmax(1))
            self.val_gts.extend(gt.detach().cpu().numpy())
        elif split == 'test':
            self.test_preds.extend(pred.detach().cpu().numpy().argmax(1))
            self.test_gts.extend(gt.detach().cpu().numpy())
        else:
            raise NotImplementedError()

    def training_epoch_end(self, outputs):
        self.evaluate_predictions(split='train')
        self.reset_metrics(split='train')

    def validation_epoch_end(self, outputs):
        self.evaluate_predictions(split='val')
        self.reset_metrics(split='val')

    def test_epoch_end(self, outputs):
        self.evaluate_predictions(split='test')
        self.reset_metrics(split='test')

    def evaluate_predictions(self, split):
        if split == 'train':
            preds = self.train_preds
            gts = self.train_gts
        elif split == 'val':
            preds = self.val_preds
            gts = self.val_gts
        elif split == 'test':
            preds = self.test_preds
            gts = self.test_gts
        else:
            raise NotImplementedError()

        cls_report = classification_report(gts, preds)
        print(split)
        print(cls_report)

# Define the data transforms
pil_transform = transforms.Compose([transforms.Resize((224, 224))])

# Define the datasets
train_dataset = LiverEndoscopy(split='train', balance_data=True, task_type='classification', temporal=False, pil_transform=pil_transform)
val_dataset = LiverEndoscopy(split='val', balance_data=True, task_type='classification', temporal=False, pil_transform=pil_transform)
test_dataset = LiverEndoscopy(split='test', balance_data=True, task_type='classification', temporal=False, pil_transform=pil_transform)

# Define the data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=1)

# Define the model
model = Classifier(model_type='resnet', num_classes=3, learning_rate=1e-3)

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Balancing data by oversampling under-represented classes...
Balancing data by oversampling under-represented classes...
Balancing data by oversampling under-represented classes...


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 157MB/s]


In [ ]:
trainer = pl.Trainer(gpus=1, max_epochs=3, num_sanity_val_steps=0)

trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNetModel      | 11.2 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.712    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: 0it [00:00, ?it/s]

val
              precision    recall  f1-score   support

           0       1.00      0.32      0.48      1200
           1       0.59      1.00      0.75      1200

    accuracy                           0.66      2400
   macro avg       0.80      0.66      0.61      2400
weighted avg       0.80      0.66      0.61      2400

train
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      3274
           1       0.98      0.99      0.99      3274

    accuracy                           0.99      6548
   macro avg       0.99      0.99      0.99      6548
weighted avg       0.99      0.99      0.99      6548



...

Validation: 0it [00:00, ?it/s]

val
              precision    recall  f1-score   support

           0       1.00      0.01      0.02      1200
           1       0.50      1.00      0.67      1200

    accuracy                           0.50      2400
   macro avg       0.75      0.50      0.34      2400
weighted avg       0.75      0.50      0.34      2400

train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3274
           1       1.00      1.00      1.00      3274

    accuracy                           1.00      6548
   macro avg       1.00      1.00      1.00      6548
weighted avg       1.00      1.00      1.00      6548



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: 0it [00:00, ?it/s]

val
              precision    recall  f1-score   support

           0       1.00      0.49      0.66      1200
           1       0.66      1.00      0.80      1200

    accuracy                           0.75      2400
   macro avg       0.83      0.75      0.73      2400
weighted avg       0.83      0.75      0.73      2400

train
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3274
           1       1.00      1.00      1.00      3274

    accuracy                           1.00      6548
   macro avg       1.00      1.00      1.00      6548
weighted avg       1.00      1.00      1.00      6548



In [ ]:
trainer.test(model, dataloaders=test_loader)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


test
              precision    recall  f1-score   support

           0       0.85      0.72      0.78      1964
           1       0.76      0.87      0.81      1964

    accuracy                           0.80      3928
   macro avg       0.80      0.80      0.79      3928
weighted avg       0.80      0.80      0.79      3928



[{}]